from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

### In the interest of saving time, for each of the models I reduced the number of steps from 20,000 to 2,000 and increased the learning rate from .001 to .01. When I tested this with Model 1, I found this only very slightly decreased the accuracy and overall f-score.

# Model 1 (tutorial example -- Two convolutional layers w/activated layers, two pooling layers, and two fully connected layers)

In [2]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  

  #Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def get_metrics(a, p):
    actual = a
    predict = p
    print("------Confusion Matrix------")
    print(confusion_matrix(actual, predict))
    
    print(classification_report(actual, predict))
    print("------Accuracy------")
    ascore = accuracy_score(actual, predict)
    print(ascore)

In [4]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model1")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=100)
    
  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=2000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    num_epochs = 1,
    shuffle=False)

  predict_one_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data[0]},
    num_epochs = 1,
    shuffle=False)

  results = list(mnist_classifier.predict(input_fn=predict_input_fn))
  predictions = [i['classes'] for i in results]
  get_metrics(eval_labels, predictions)

In [5]:
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9a42cec110>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_model1', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model1/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.09741955  0.10201871  0.09542349  0.10320755  0.08905

INFO:tensorflow:loss = 2.31143, step = 1
INFO:tensorflow:global_step/sec: 1.12598
INFO:tensorflow:probabilities = [[ 0.09872715  0.07285458  0.08930586  0.1450178   0.10073947  0.11292462
   0.0722016   0.10853685  0.09767018  0.10202199]
 [ 0.1044966   0.08145261  0.10193969  0.11429317  0.089247    0.11850135
   0.09880022  0.08386768  0.10425447  0.10314715]
 [ 0.07655136  0.15698345  0.10036411  0.10849516  0.07374337  0.07123851
   0.08456214  0.10322642  0.12940909  0.0954264 ]
 [ 0.09383905  0.06834047  0.12328311  0.09856781  0.13026893  0.07728535
   0.11640099  0.10704739  0.10016955  0.08479742]
 [ 0.09124294  0.081398    0.1219706   0.1164634   0.08565827  0.09403489
   0.141553    0.08297313  0.09327302  0.09143274]
 [ 0.09033664  0.07249298  0.09865417  0.08828636  0.08274687  0.1040238
   0.09236752  0.14592193  0.12060273  0.10456702]
 [ 0.18485466  0.0569149   0.12662256  0.09740957  0.09163015  0.08113874
   0.08737248  0.07648595  0.11701924  0.08055184]
 [ 0.1006233

INFO:tensorflow:loss = 2.01558, step = 101 (88.837 sec)
INFO:tensorflow:global_step/sec: 1.23019
INFO:tensorflow:probabilities = [[ 0.00692213  0.02863317  0.0668096   0.01751     0.09264886  0.03629217
   0.65132415  0.00030083  0.05627614  0.04328297]
 [ 0.02043429  0.43423793  0.05735275  0.01048804  0.04516979  0.14626062
   0.01025467  0.01577723  0.2222632   0.03776157]
 [ 0.18230809  0.03020502  0.02877657  0.02418153  0.19268399  0.13432646
   0.02150438  0.15520613  0.12158772  0.10922013]
 [ 0.00186798  0.35492334  0.01543897  0.11307114  0.03742201  0.04920873
   0.06881727  0.0712005   0.21006228  0.07798775]
 [ 0.03704533  0.00565595  0.02501284  0.08956766  0.00572732  0.28942046
   0.00641948  0.01039727  0.51990277  0.01085094]
 [ 0.00901552  0.00227447  0.00108202  0.0154513   0.1713358   0.0280276
   0.00330645  0.32789117  0.00414306  0.43747255]
 [ 0.19584121  0.00433788  0.0205156   0.1921639   0.0274418   0.24320845
   0.11066046  0.01386379  0.15135364  0.0406132

INFO:tensorflow:loss = 0.782957, step = 201 (81.282 sec)
INFO:tensorflow:global_step/sec: 1.11244
INFO:tensorflow:probabilities = [[ 0.00006952  0.00000004  0.000163    0.00001367  0.94639021  0.00028557
   0.00007975  0.0003181   0.00031184  0.05236828]
 [ 0.001104    0.00413147  0.00054313  0.00215525  0.00062998  0.00502834
   0.00001565  0.94321084  0.00683999  0.03634141]
 [ 0.00232088  0.01385977  0.01225413  0.00309112  0.72492266  0.00942923
   0.03002576  0.03448468  0.03156311  0.1380486 ]
 [ 0.00034973  0.91346562  0.00408048  0.02258913  0.00179387  0.00345716
   0.00516148  0.00900378  0.0238893   0.0162095 ]
 [ 0.00029988  0.82698816  0.00264609  0.00250084  0.00018331  0.00329272
   0.00136627  0.00539403  0.15132715  0.00600159]
 [ 0.0001614   0.00573936  0.00275906  0.00318551  0.20808694  0.01154463
   0.02608873  0.00391385  0.06791447  0.67060602]
 [ 0.00011961  0.00006755  0.0004233   0.00054805  0.00015604  0.0027069
   0.00001421  0.92871374  0.00552671  0.061723

INFO:tensorflow:loss = 0.33331, step = 301 (89.875 sec)
INFO:tensorflow:global_step/sec: 1.19523
INFO:tensorflow:probabilities = [[ 0.00009495  0.00000031  0.0000015   0.00005143  0.56615764  0.00040531
   0.00018837  0.00037397  0.00087974  0.43184686]
 [ 0.00091477  0.00001368  0.00064637  0.0001655   0.9484511   0.00163886
   0.00314993  0.0013365   0.00176053  0.0419227 ]
 [ 0.00072726  0.00375995  0.00102569  0.08226798  0.00001858  0.89949024
   0.00325156  0.00024447  0.00908808  0.00012619]
 [ 0.00046474  0.00014009  0.00004049  0.0039433   0.03565047  0.02061917
   0.00095464  0.00813855  0.01345976  0.91658872]
 [ 0.00785378  0.0067973   0.03247219  0.05256883  0.00000121  0.00554222
   0.0001907   0.00074409  0.89371639  0.00011328]
 [ 0.00001339  0.0000054   0.00004801  0.00017312  0.96107638  0.00028045
   0.0004478   0.00019519  0.00149226  0.03626798]
 [ 0.00376575  0.01144836  0.00124338  0.04155187  0.2034734   0.01854987
   0.00431418  0.01772537  0.06658036  0.631347

...

INFO:tensorflow:loss = 0.153429, step = 1701 (80.722 sec)
INFO:tensorflow:global_step/sec: 1.23101
INFO:tensorflow:probabilities = [[ 0.00021203  0.00082247  0.08528529  0.59551466  0.00110105  0.06846669
   0.00013342  0.01016514  0.20905972  0.02923947]
 [ 0.00001172  0.00000001  0.00000006  0.00000033  0.00000397  0.0000858
   0.          0.99753511  0.00000093  0.00236201]
 [ 0.00116939  0.96632695  0.00443286  0.00284239  0.00096474  0.0004048
   0.00043469  0.00487449  0.01798782  0.00056178]
 [ 0.00003482  0.00000228  0.99929011  0.00065632  0.00000034  0.00000258
   0.00000422  0.00000403  0.00000513  0.00000018]
 [ 0.99976987  0.00000004  0.00016815  0.0000053   0.00000076  0.00000152
   0.00000106  0.00000281  0.00003484  0.00001569]
 [ 0.0000135   0.00000278  0.00020801  0.00000331  0.00021651  0.00010128
   0.9994204   0.00000002  0.00003215  0.00000207]
 [ 0.00022794  0.00002984  0.00021859  0.07626399  0.00000447  0.00010939
   0.00000032  0.00000796  0.92271256  0.000424

INFO:tensorflow:loss = 0.0768114, step = 1801 (81.226 sec)
INFO:tensorflow:global_step/sec: 1.13634
INFO:tensorflow:probabilities = [[ 0.00003865  0.99908113  0.00005955  0.00011843  0.00001968  0.0000084
   0.00000485  0.00052554  0.00011718  0.00002651]
 [ 0.00073453  0.00000076  0.00001227  0.00028234  0.00000028  0.99723887
   0.00018601  0.00000006  0.00154     0.00000485]
 [ 0.00000335  0.00000071  0.00543163  0.00006119  0.99432194  0.00000171
   0.00007361  0.0000031   0.00000294  0.00009985]
 [ 0.00000118  0.00000301  0.99654597  0.00262368  0.00004975  0.00000139
   0.00000039  0.00006062  0.00000435  0.00070962]
 [ 0.00000117  0.00001671  0.00000743  0.00003968  0.98439425  0.00043811
   0.00001839  0.00018848  0.00012917  0.01476667]
 [ 0.00054547  0.0000064   0.00222838  0.00000113  0.00062676  0.00016559
   0.9963659   0.00000134  0.00004045  0.00001861]
 [ 0.00004811  0.00008612  0.00127352  0.01193763  0.02317381  0.02872423
   0.00012674  0.00778304  0.00793779  0.9189

INFO:tensorflow:loss = 0.137443, step = 1901 (88.013 sec)
INFO:tensorflow:Saving checkpoints for 1983 into /tmp/mnist_convnet_model1/model.ckpt.
INFO:tensorflow:Saving checkpoints for 2000 into /tmp/mnist_convnet_model1/model.ckpt.
INFO:tensorflow:Loss for final step: 0.132124.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model1/model.ckpt-2000
------Confusion Matrix------
[[ 972    0    1    0    0    3    1    1    2    0]
 [   0 1123    4    1    1    0    1    0    5    0]
 [   6    1 1002    5    4    1    0    7    6    0]
 [   0    0    6  977    0   11    0    7    7    2]
 [   1    0    3    0  965    0    2    1    2    8]
 [   3    0    1    7    1  871    6    1    1    1]
 [   9    4    1    0    9    7  925    0    3    0]
 [   2    6   17    4    4    0    0  978    3   14]
 [   7    0    3    7    4    5    5    4  936    3]
 [   7    7    2   11   18    6    0    4    4  950]]
             precision    recall  f1-score   support

          0       0.97 

SystemExit: 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Model 2 - One convolution layer w/activation layer and one pooling layer

In [6]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  
  # Dense Layer
  pool1_flat = tf.reshape(pool1, [-1, 14 * 14 * 32])

  dense = tf.layers.dense(inputs=pool1_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
  

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

In [7]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def get_metrics(a, p):
    actual = a
    predict = p
    print("------Confusion Matrix------")
    print(confusion_matrix(actual, predict))
    
    print(classification_report(actual, predict))
    print("------Accuracy------")
    ascore = accuracy_score(actual, predict)
    print(ascore)

In [8]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model2")

  # Set up logging for predictions
 #Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)
 
  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=200,
      num_epochs=None,
      shuffle=True)
  
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=2000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    num_epochs = 1,
    shuffle=False)

  results = list(mnist_classifier.predict(input_fn=predict_input_fn))
  predictions = [i['classes'] for i in results]
  get_metrics(eval_labels, predictions)
  
  exit

In [9]:
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9a40f502d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_model2', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model2/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.1124995   0.12428655  0.09900792 ...,  0.09456603  0.

INFO:tensorflow:loss = 0.354891, step = 1101 (95.884 sec)
INFO:tensorflow:global_step/sec: 1.21462
INFO:tensorflow:probabilities = [[ 0.00088126  0.00007095  0.00104673 ...,  0.00000045  0.0005224
   0.0000041 ]
 [ 0.00180531  0.000079    0.00420156 ...,  0.00033911  0.93718231
   0.01361529]
 [ 0.00030205  0.0000773   0.00046991 ...,  0.00334537  0.15169565
   0.1666476 ]
 ..., 
 [ 0.00080534  0.00001349  0.00030049 ...,  0.0000154   0.0068864
   0.00025439]
 [ 0.00006442  0.0000094   0.01440325 ...,  0.00502461  0.00463456
   0.95971304]
 [ 0.00003889  0.00000027  0.00000016 ...,  0.99014246  0.00000992
   0.00938186]] (82.328 sec)
INFO:tensorflow:loss = 0.22463, step = 1201 (82.330 sec)
INFO:tensorflow:global_step/sec: 1.23525
INFO:tensorflow:probabilities = [[ 0.00187904  0.00014148  0.00005946 ...,  0.00012905  0.02429453
   0.02414074]
 [ 0.00054361  0.00000274  0.01082116 ...,  0.00000127  0.00162815
   0.00019278]
 [ 0.00006906  0.00000111  0.00012924 ...,  0.00066081  0.014625

SystemExit: 

# Model 3 -- Two convolutional layers followed by one pooling layer

In [10]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters = 16,
      kernel_size=[5, 5],
      padding="valid",
      activation=tf.nn.relu)
    
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  
  # Dense Layer
  pool1_flat = tf.reshape(pool1, [-1, 12 * 12 * 16])

  dense = tf.layers.dense(inputs=pool1_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)
    
  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  

In [11]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def get_metrics(a, p):
    actual = a
    predict = p
    print("------Confusion Matrix------")
    print(confusion_matrix(actual, predict))
    
    print(classification_report(actual, predict))
    print("------Accuracy------")
    ascore = accuracy_score(actual, predict)
    print(ascore)

In [12]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model3")

  # Set up logging for predictions
 #Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)
 
  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=200,
      num_epochs=None,
      shuffle=True)
  
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=2000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    num_epochs = 1,
    shuffle=False)

  results = list(mnist_classifier.predict(input_fn=predict_input_fn))
  predictions = [i['classes'] for i in results]
  get_metrics(eval_labels, predictions)

In [13]:
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9a40f106d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_model3', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model3/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.08519147  0.11078481  0.0902482  ...,  0.10921299  0.

INFO:tensorflow:probabilities = [[ 0.00118331  0.00172816  0.02255215 ...,  0.00379812  0.30956513
   0.11382187]
 [ 0.00000144  0.00000012  0.00004213 ...,  0.99046201  0.00002648
   0.0093162 ]
 [ 0.00047419  0.00079289  0.0001854  ...,  0.00054057  0.00062971
   0.00387359]
 ..., 
 [ 0.0000336   0.00050511  0.00216483 ...,  0.12938695  0.01450995
   0.54735607]
 [ 0.00000345  0.00000036  0.99947232 ...,  0.00000013  0.00000004
   0.00000039]
 [ 0.0000276   0.00089874  0.99502689 ...,  0.00000004  0.00308739
   0.00000008]] (230.171 sec)
INFO:tensorflow:loss = 0.236648, step = 1101 (230.169 sec)
INFO:tensorflow:Saving checkpoints for 1124 into /tmp/mnist_convnet_model3/model.ckpt.
INFO:tensorflow:global_step/sec: 0.466419
INFO:tensorflow:probabilities = [[ 0.00000009  0.          0.00000116 ...,  0.00000251  0.00000111
   0.0001831 ]
 [ 0.00000147  0.00000001  0.00035437 ...,  0.99956352  0.00001106
   0.00005493]
 [ 0.99998415  0.00000001  0.00000008 ...,  0.0000032   0.00000038
   

SystemExit: 

# Initial Results
Model 1: Accuracy 96.99%, F-Score .97

Model 2: Accuracy 95.06%, F-Score .95

Model 3: Accuracy 96.68, F-Score .97

The highest accuracy model I found was Model 1 (96.99%), which was just barely better than Model 3 (96.68%). However, model 1 runs significantly faster than Model 3 (approximately 90 seconds per 100 steps, vs. approximately 250 seconds per 100 steps). The pooling layer didn't seem to have a significant effect on the accuracy of the model, but it allowed it to run more quickly by reducing the inputs received by the second convolution layer. While I did adjust some of the other hyperparameters in models 2 and 3, I suspect the changes to the layers had the most significant effect on the differences between model 2 and the other models. The pooling layer may not have significantly affected the accuracy of the model, but the second convolution layer did (this makes since, given the calculations that happen in the convolution model).

The following four models are variations on the 2 convolution layer/two pooling layer model (model 1), with adjustments to the kernel, filters, dropout rates, and batch size respectively

# Model 1A - 3x3 kernel

In [14]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  

  #Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  

In [15]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def get_metrics(a, p):
    actual = a
    predict = p
    print("------Confusion Matrix------")
    print(confusion_matrix(actual, predict))
    
    print(classification_report(actual, predict))
    print("------Accuracy------")
    ascore = accuracy_score(actual, predict)
    print(ascore)

In [16]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model1a")

  # Set up logging for predictions
 #Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)
 
  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=2000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    num_epochs = 1,
    shuffle=False)

  results = list(mnist_classifier.predict(input_fn=predict_input_fn))
  predictions = [i['classes'] for i in results]
  get_metrics(eval_labels, predictions)

In [17]:
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9a40ead090>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_model1a', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model1a/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.10492875  0.10141823  0.09682251  0.0964691   0.105

INFO:tensorflow:loss = 2.30336, step = 1
INFO:tensorflow:global_step/sec: 2.13294
INFO:tensorflow:probabilities = [[ 0.10389438  0.07738797  0.10064766  0.09881693  0.11188386  0.08291625
   0.09948593  0.1177605   0.09950832  0.10769822]
 [ 0.09495015  0.09648381  0.09516097  0.1038091   0.10093085  0.08143528
   0.08643368  0.11216394  0.10524169  0.12339057]
 [ 0.0895146   0.08091586  0.09296417  0.08189617  0.10723139  0.08591493
   0.12989578  0.09656854  0.10699239  0.12810621]
 [ 0.12621093  0.08000871  0.11915133  0.09982392  0.07761279  0.07940041
   0.1134232   0.08251521  0.11470614  0.10714748]
 [ 0.12565492  0.10402279  0.08276628  0.10814951  0.07767247  0.11073346
   0.08440096  0.07215104  0.13275035  0.10169824]
 [ 0.0880971   0.08726414  0.11744652  0.11402066  0.07755131  0.10183359
   0.09354804  0.10937585  0.11995521  0.0909076 ]
 [ 0.0896356   0.07904924  0.11145408  0.11663102  0.08995848  0.07918498
   0.11973918  0.09389541  0.1053218   0.11513022]
 [ 0.094585

INFO:tensorflow:loss = 2.06917, step = 101 (46.908 sec)
INFO:tensorflow:global_step/sec: 2.2116
INFO:tensorflow:probabilities = [[ 0.04628489  0.00531947  0.11032718  0.09324539  0.13495779  0.07314339
   0.40968555  0.00643171  0.06139351  0.05921121]
 [ 0.02529418  0.0006921   0.17279133  0.01558803  0.0298803   0.04050368
   0.68535107  0.00126992  0.00391754  0.02471184]
 [ 0.04508924  0.18686132  0.05973125  0.05908779  0.04004626  0.1371616
   0.01472904  0.08904403  0.32128739  0.04696204]
 [ 0.15094878  0.00121939  0.68972749  0.0901109   0.0009428   0.01358618
   0.02072575  0.00140955  0.02858737  0.00274177]
 [ 0.05237171  0.05517051  0.16615404  0.34080029  0.02030701  0.15859576
   0.06973923  0.00304649  0.11365154  0.02016345]
 [ 0.08268709  0.00184381  0.02273323  0.29759276  0.01192453  0.25551641
   0.01226458  0.06348929  0.20504318  0.04690502]
 [ 0.22868173  0.08838115  0.13279821  0.25816223  0.01967111  0.13540338
   0.0209391   0.01791088  0.07820643  0.01984575

INFO:tensorflow:loss = 1.09444, step = 201 (45.240 sec)
INFO:tensorflow:global_step/sec: 2.16846
INFO:tensorflow:probabilities = [[ 0.00004032  0.00000337  0.0000029   0.00148683  0.00061246  0.00002388
   0.00000299  0.99238139  0.0003463   0.00509953]
 [ 0.00538619  0.00016186  0.00287301  0.00413867  0.58509815  0.01212725
   0.01538154  0.00892845  0.00283123  0.36307368]
 [ 0.00083728  0.02361152  0.8892855   0.06225083  0.00010166  0.00239806
   0.01792756  0.00018859  0.00267025  0.00072874]
 [ 0.00782224  0.00022736  0.00109437  0.0004985   0.80358118  0.0179509
   0.00663368  0.03044436  0.03148505  0.10026229]
 [ 0.00107847  0.8919822   0.01014749  0.01500467  0.00250921  0.00906555
   0.00889684  0.00731821  0.03828506  0.01571223]
 [ 0.01568014  0.00022409  0.24699804  0.58809561  0.01731855  0.02754296
   0.02551399  0.01174338  0.05909239  0.00779086]
 [ 0.00239474  0.00003291  0.00028977  0.00033257  0.88498199  0.00861498
   0.00127086  0.01804418  0.00767905  0.0763590

INFO:tensorflow:loss = 0.727501, step = 301 (46.090 sec)
INFO:tensorflow:global_step/sec: 2.22578
INFO:tensorflow:probabilities = [[ 0.95542371  0.0000018   0.00395724  0.01553425  0.00000702  0.01741836
   0.00002795  0.00026736  0.00655451  0.00080772]
 [ 0.00096305  0.00003005  0.00017322  0.00010502  0.02742546  0.00176207
   0.00006251  0.09067192  0.02121916  0.85758752]
 [ 0.98736364  0.00000099  0.00050188  0.00100561  0.00000336  0.01066687
   0.00003208  0.00000963  0.00031988  0.00009612]
 [ 0.00070654  0.00058834  0.00013357  0.00073721  0.08752367  0.00076528
   0.00188848  0.01350109  0.00815985  0.88599598]
 [ 0.00002419  0.0000011   0.00127146  0.99841166  0.0000002   0.00026223
   0.00000195  0.00000215  0.00002449  0.00000059]
 [ 0.00015125  0.00029796  0.00005198  0.00977712  0.00027946  0.00162381
   0.00001409  0.96451956  0.00040781  0.02287694]
 [ 0.00142979  0.00009657  0.0006062   0.00189621  0.11902872  0.00447783
   0.00659615  0.00101284  0.00270757  0.86214

...

INFO:tensorflow:loss = 0.24237, step = 1601 (45.745 sec)
INFO:tensorflow:global_step/sec: 2.11227
INFO:tensorflow:probabilities = [[ 0.00034497  0.00004449  0.00023955  0.96249491  0.00000039  0.02923711
   0.00000054  0.00001644  0.00749254  0.00012922]
 [ 0.00000121  0.00000036  0.00006491  0.00002547  0.00000006  0.00000048
   0.          0.99960834  0.00000318  0.00029608]
 [ 0.00000003  0.00000003  0.00000126  0.99762952  0.          0.00223413
   0.          0.00000013  0.00013095  0.00000391]
 [ 0.00048994  0.00006508  0.98194522  0.00328398  0.00000001  0.00011255
   0.00000016  0.00017043  0.01389498  0.00003757]
 [ 0.00417944  0.00000003  0.00000007  0.00004569  0.0008949   0.00673933
   0.00000036  0.98633933  0.00002886  0.00177191]
 [ 0.0001245   0.985843    0.00357902  0.00133645  0.00040098  0.00000997
   0.0000284   0.00117048  0.00728753  0.00021962]
 [ 0.00144536  0.00006813  0.00123254  0.00004071  0.01213604  0.00040167
   0.9837482   0.00001598  0.00086309  0.00004

INFO:tensorflow:loss = 0.3112, step = 1701 (47.338 sec)
INFO:tensorflow:global_step/sec: 2.15312
INFO:tensorflow:probabilities = [[ 0.00000191  0.00001097  0.00020784  0.0001428   0.00000949  0.00001847
   0.00000001  0.99254483  0.00050604  0.00655761]
 [ 0.00000066  0.00000074  0.00252686  0.99733788  0.00000002  0.00009972
   0.00000009  0.00000037  0.00003154  0.00000211]
 [ 0.00211999  0.00000527  0.0035537   0.00000148  0.03889227  0.00048597
   0.95493472  0.00000239  0.00000405  0.0000003 ]
 [ 0.98817283  0.00000094  0.0009739   0.0000463   0.00002671  0.00824018
   0.000136    0.00027901  0.00091958  0.00120461]
 [ 0.00000067  0.00022537  0.99463058  0.0027906   0.00007603  0.00019035
   0.00000441  0.00017393  0.00157505  0.00033299]
 [ 0.00001819  0.00001676  0.00001337  0.00306089  0.00000527  0.99641621
   0.00000048  0.00003823  0.00000888  0.00042176]
 [ 0.00000016  0.00004966  0.99932981  0.00056914  0.00000001  0.00000048
   0.00000167  0.00004883  0.00000034  0.000000

INFO:tensorflow:loss = 0.169542, step = 1801 (46.450 sec)
INFO:tensorflow:global_step/sec: 2.10355
INFO:tensorflow:probabilities = [[ 0.00565923  0.00108302  0.00041079  0.00122427  0.00022813  0.0049193
   0.98608124  0.00000689  0.00036619  0.00002091]
 [ 0.00015851  0.99237388  0.00058057  0.00022058  0.00017301  0.00012506
   0.00008885  0.00077412  0.0052928   0.0002127 ]
 [ 0.00001359  0.00031954  0.00129553  0.99047172  0.00000111  0.00258552
   0.00001836  0.00001466  0.00526505  0.00001491]
 [ 0.99999666  0.          0.00000112  0.00000008  0.          0.00000015
   0.00000005  0.00000001  0.00000019  0.00000171]
 [ 0.0000656   0.00000089  0.00002931  0.00001399  0.99913788  0.000015
   0.00002029  0.00005479  0.00002406  0.00063811]
 [ 0.29957536  0.00000294  0.00062731  0.00001855  0.00085756  0.00034589
   0.69799984  0.00000857  0.00026146  0.00030256]
 [ 0.00001292  0.99458683  0.00113175  0.00062083  0.00055608  0.00000718
   0.00000707  0.00122197  0.0017392   0.0001160

INFO:tensorflow:loss = 0.0735642, step = 1901 (47.557 sec)
INFO:tensorflow:Saving checkpoints for 2000 into /tmp/mnist_convnet_model1a/model.ckpt.
INFO:tensorflow:Loss for final step: 0.158465.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model1a/model.ckpt-2000
------Confusion Matrix------
[[ 962    0    1    0    0    2   11    1    3    0]
 [   0 1118    4    1    0    0    4    0    8    0]
 [   9    0  982   10    5    0    6    7   12    1]
 [   1    1    8  952    0   19    0    7   16    6]
 [   1    0    3    0  939    0   11    2    2   24]
 [   4    1    0    6    0  860   11    1    6    3]
 [   5    3    1    0    4    8  934    1    2    0]
 [   1    4   19    2    2    0    0  973    2   25]
 [   6    0    3    7    3   10    9    3  924    9]
 [   5    7    1    8   11    4    1    6    6  960]]
             precision    recall  f1-score   support

          0       0.97      0.98      0.97       980
          1       0.99      0.99      0.99      1135
 

SystemExit: 

# Model 1B - More filters on the first convolution layer

In [18]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  

  #Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  

In [19]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def get_metrics(a, p):
    actual = a
    predict = p
    print("------Confusion Matrix------")
    print(confusion_matrix(actual, predict))
    
    print(classification_report(actual, predict))
    print("------Accuracy------")
    ascore = accuracy_score(actual, predict)
    print(ascore)

In [20]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model1b")

  # Set up logging for predictions
 #Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)
 
  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=2000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    num_epochs = 1,
    shuffle=False)

  results = list(mnist_classifier.predict(input_fn=predict_input_fn))
  predictions = [i['classes'] for i in results]
  get_metrics(eval_labels, predictions)

In [21]:
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9a42a4de50>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_model1b', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model1b/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.09910178  0.09421255  0.10049325  0.09466946  0.108

INFO:tensorflow:loss = 2.30576, step = 1
INFO:tensorflow:global_step/sec: 1.0729
INFO:tensorflow:probabilities = [[ 0.10944086  0.07471722  0.11097246  0.11433344  0.09411141  0.1049842
   0.10406106  0.10959868  0.08235506  0.09542567]
 [ 0.07964036  0.08722439  0.11074212  0.09391016  0.11551031  0.07938494
   0.09388009  0.12316709  0.09853981  0.1180008 ]
 [ 0.12026842  0.06628801  0.12222099  0.10496628  0.08827341  0.10709433
   0.12447283  0.08320659  0.08786738  0.09534173]
 [ 0.11542447  0.0770013   0.10195785  0.0905307   0.1073668   0.08501282
   0.1051129   0.10565532  0.09804513  0.11389271]
 [ 0.09719203  0.09320208  0.10532248  0.11662988  0.09484288  0.08959321
   0.11839776  0.09274312  0.09997345  0.09210318]
 [ 0.09593968  0.07235476  0.08728201  0.09022315  0.15031518  0.09496722
   0.09357591  0.12860207  0.09128009  0.09545989]
 [ 0.09718599  0.09676383  0.09959838  0.09885646  0.11418935  0.08806206
   0.09844793  0.09644303  0.11248612  0.09796687]
 [ 0.10953493

INFO:tensorflow:loss = 2.14999, step = 101 (93.173 sec)
INFO:tensorflow:global_step/sec: 0.972683
INFO:tensorflow:probabilities = [[ 0.03475744  0.16415121  0.05541695  0.10628744  0.05548395  0.08539057
   0.10862686  0.04817344  0.27869692  0.06301522]
 [ 0.04358815  0.06346804  0.0829242   0.12384555  0.03023272  0.06522779
   0.39114833  0.03743177  0.08159016  0.08054329]
 [ 0.07337841  0.06907534  0.04204036  0.02567361  0.25393909  0.13175802
   0.09900436  0.07211413  0.15733317  0.07568344]
 [ 0.04706638  0.03146647  0.10255732  0.07603039  0.15919496  0.05337615
   0.09332541  0.15123047  0.18334123  0.1024112 ]
 [ 0.01934038  0.03997722  0.01403714  0.07029027  0.05132298  0.04939046
   0.02086506  0.53999317  0.08450061  0.11028267]
 [ 0.81207192  0.00122924  0.01523143  0.0530424   0.00825261  0.03884709
   0.01625989  0.01409659  0.02032032  0.02064835]
 [ 0.02767823  0.06218055  0.01366178  0.0597596   0.13902292  0.03929737
   0.01724696  0.38389045  0.10842071  0.14884

INFO:tensorflow:loss = 1.23943, step = 201 (102.835 sec)
INFO:tensorflow:global_step/sec: 1.0121
INFO:tensorflow:probabilities = [[ 0.01653271  0.0006489   0.00040781  0.00300746  0.04566327  0.76100534
   0.00281285  0.03364877  0.11061697  0.0256559 ]
 [ 0.00961574  0.00053802  0.00013667  0.00225394  0.13167416  0.02027139
   0.00078976  0.13763808  0.14042662  0.55665559]
 [ 0.01878405  0.08458788  0.00061992  0.02195695  0.04849285  0.02431995
   0.02519761  0.4270114   0.10632376  0.24270552]
 [ 0.00052278  0.89701116  0.0261178   0.01712149  0.00134764  0.00298362
   0.00807379  0.00151357  0.04223812  0.00307003]
 [ 0.04177583  0.0096541   0.00241986  0.01460733  0.0310085   0.65725911
   0.0039579   0.03993658  0.17993551  0.01944529]
 [ 0.00178883  0.04591523  0.00071453  0.03184572  0.14433686  0.02848198
   0.00732333  0.22322026  0.07026239  0.4461109 ]
 [ 0.08709086  0.00003636  0.02899623  0.09667792  0.0879451   0.13457449
   0.03474931  0.01464296  0.49732307  0.017963

INFO:tensorflow:loss = 0.568818, step = 301 (98.783 sec)
INFO:tensorflow:global_step/sec: 1.33848
INFO:tensorflow:probabilities = [[ 0.97546571  0.00000115  0.00040984  0.00122321  0.00014533  0.01861355
   0.00279538  0.000053    0.00101936  0.00027338]
 [ 0.01044644  0.00880855  0.04088875  0.00431257  0.007953    0.02268641
   0.83934081  0.00021618  0.0642071   0.00114016]
 [ 0.00044504  0.00003712  0.00710203  0.00135931  0.2770938   0.00117161
   0.00230426  0.02845713  0.00785585  0.67417389]
 [ 0.00074174  0.0030644   0.80763233  0.0020291   0.03020646  0.00068563
   0.00981587  0.00104991  0.03079572  0.11397882]
 [ 0.92438722  0.00000012  0.0009467   0.02351145  0.00001708  0.03907932
   0.00003004  0.00235672  0.00690852  0.00276282]
 [ 0.00020426  0.77722985  0.02295358  0.09442442  0.00584857  0.02213298
   0.00665953  0.01408319  0.03118825  0.02527545]
 [ 0.02488058  0.00367662  0.0351195   0.29933202  0.00017077  0.07456525
   0.55348712  0.00009878  0.00803725  0.00063

...

INFO:tensorflow:loss = 0.158822, step = 1601 (76.234 sec)
INFO:tensorflow:global_step/sec: 1.27988
INFO:tensorflow:probabilities = [[ 0.00053877  0.00032053  0.82992911  0.00427437  0.1612294   0.00131496
   0.00135894  0.00003789  0.00072658  0.00026937]
 [ 0.00016462  0.23548883  0.00563595  0.37151191  0.12482934  0.09490134
   0.00081919  0.00781528  0.09922571  0.05960783]
 [ 0.00000028  0.00024356  0.99927145  0.00048023  0.00000003  0.0000001
   0.00000046  0.          0.00000375  0.00000002]
 [ 0.00055704  0.00000003  0.00000239  0.000051    0.00005765  0.00006734
   0.00000032  0.98288083  0.00009147  0.01629199]
 [ 0.0000034   0.00012358  0.00027306  0.99775475  0.0000048   0.00116203
   0.00000043  0.00000432  0.00066147  0.00001216]
 [ 0.00002156  0.00000011  0.00007917  0.00000024  0.00006289  0.00026168
   0.99957067  0.          0.00000364  0.0000001 ]
 [ 0.00044393  0.          0.00000044  0.00001217  0.00000271  0.0000532
   0.00000002  0.99737728  0.00000013  0.002110

INFO:tensorflow:loss = 0.12462, step = 1701 (78.129 sec)
INFO:tensorflow:global_step/sec: 1.29376
INFO:tensorflow:probabilities = [[ 0.00024054  0.00001185  0.00019908  0.00043803  0.0001996   0.00006903
   0.00002178  0.29702926  0.00002752  0.70176333]
 [ 0.00002392  0.00004693  0.0003319   0.00247758  0.02365202  0.00062254
   0.00001145  0.00760326  0.00025525  0.96497512]
 [ 0.00091004  0.00016431  0.00012199  0.97402984  0.00002063  0.02324378
   0.00000316  0.00028767  0.00092351  0.00029513]
 [ 0.00000019  0.00009422  0.00018799  0.00589289  0.00730055  0.00049773
   0.00000139  0.00974229  0.00027895  0.97600377]
 [ 0.00001461  0.00000028  0.00000369  0.00046632  0.00000022  0.00007615
   0.          0.99893206  0.0000013   0.00050525]
 [ 0.99961156  0.00000006  0.00000164  0.0001116   0.00000016  0.00025944
   0.00000185  0.00001144  0.00000167  0.00000076]
 [ 0.00004539  0.002563    0.00109849  0.00030715  0.01045661  0.05847395
   0.92321271  0.00000088  0.00380335  0.00003

INFO:tensorflow:loss = 0.189614, step = 1801 (77.324 sec)
INFO:tensorflow:global_step/sec: 1.33978
INFO:tensorflow:probabilities = [[ 0.99016833  0.00000037  0.00047983  0.00002495  0.          0.00889622
   0.00000537  0.00033777  0.00008312  0.00000394]
 [ 0.00002224  0.0000001   0.00000678  0.00002623  0.0343213   0.00003475
   0.00000322  0.00599647  0.00519411  0.95439488]
 [ 0.99588543  0.00000005  0.00172276  0.00000057  0.00000032  0.00030198
   0.0020567   0.00001137  0.0000125   0.00000823]
 [ 0.00000144  0.          0.00001067  0.00000124  0.99983191  0.00000725
   0.00008637  0.00000044  0.00000164  0.00005903]
 [ 0.00659338  0.00006062  0.9824754   0.00478172  0.00004105  0.00066371
   0.00009721  0.00108334  0.00329499  0.00090852]
 [ 0.00481791  0.00002125  0.00169505  0.00001944  0.00110477  0.00201656
   0.99011737  0.00000842  0.00017981  0.00001941]
 [ 0.00006031  0.00000131  0.00003816  0.00000139  0.00006937  0.0001768
   0.99942112  0.00000003  0.00023128  0.00000

INFO:tensorflow:loss = 0.233751, step = 1901 (74.611 sec)
INFO:tensorflow:Saving checkpoints for 2000 into /tmp/mnist_convnet_model1b/model.ckpt.
INFO:tensorflow:Loss for final step: 0.22427.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model1b/model.ckpt-2000
------Confusion Matrix------
[[ 967    1    1    1    0    0    7    1    2    0]
 [   0 1119    4    1    0    0    4    0    7    0]
 [   7    0  993    6    3    0    3    9   11    0]
 [   0    1    7  984    0    1    0    7    8    2]
 [   1    0    5    0  949    0    7    3    2   15]
 [   5    1    1   28    2  837   10    1    5    2]
 [   7    3    4    2    4    6  929    1    2    0]
 [   1    6   17    4    1    0    0  992    2    5]
 [   5    2    7   18    4    4   10    5  916    3]
 [  10    6    2   16   22    1    1    9    5  937]]
             precision    recall  f1-score   support

          0       0.96      0.99      0.98       980
          1       0.98      0.99      0.98      1135
   

SystemExit: 

# Model 1C -- reducing dropout rate

In [22]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  

  #Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
    
  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  

In [23]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def get_metrics(a, p):
    actual = a
    predict = p
    print("------Confusion Matrix------")
    print(confusion_matrix(actual, predict))
    
    print(classification_report(actual, predict))
    print("------Accuracy------")
    ascore = accuracy_score(actual, predict)
    print(ascore)

In [24]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model1c")

  # Set up logging for predictions
 #Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)
 
  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=2000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    num_epochs = 1,
    shuffle=False)

  results = list(mnist_classifier.predict(input_fn=predict_input_fn))
  predictions = [i['classes'] for i in results]
  get_metrics(eval_labels, predictions)

In [25]:
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9a3fffb590>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_model1c', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model1c/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.08868694  0.09709741  0.09502432  0.0965636   0.097

INFO:tensorflow:loss = 2.29309, step = 1
INFO:tensorflow:global_step/sec: 2.21475
INFO:tensorflow:probabilities = [[ 0.10000397  0.10305796  0.0985347   0.09950261  0.08534327  0.10166151
   0.10714582  0.08794484  0.11530549  0.10149986]
 [ 0.10908798  0.07265265  0.09736794  0.10365377  0.09437513  0.1105613
   0.10215109  0.0933359   0.11238268  0.10443159]
 [ 0.10452898  0.0887334   0.09668249  0.11070221  0.08875723  0.10900209
   0.09644786  0.09002032  0.11046959  0.10465579]
 [ 0.11091166  0.07725125  0.09602614  0.13297977  0.07084613  0.11857025
   0.09559509  0.08728226  0.1219802   0.08855732]
 [ 0.09311002  0.07681218  0.09800432  0.08201584  0.10846985  0.09061932
   0.09792523  0.10135543  0.12896827  0.12271963]
 [ 0.08860001  0.12747143  0.08782291  0.09716068  0.09317187  0.1102299
   0.10196651  0.08893037  0.10411774  0.10052865]
 [ 0.10395316  0.0742891   0.10645838  0.09793764  0.10403883  0.1174363
   0.1092786   0.08372821  0.11531085  0.08756892]
 [ 0.10346416 

INFO:tensorflow:loss = 2.13347, step = 101 (45.172 sec)
INFO:tensorflow:global_step/sec: 2.26821
INFO:tensorflow:probabilities = [[ 0.0115972   0.0967763   0.01713659  0.11625949  0.1041224   0.05446995
   0.0227512   0.27231443  0.1355765   0.16899596]
 [ 0.14055091  0.02511098  0.08977714  0.18833365  0.03325543  0.16559751
   0.12812884  0.06604355  0.07890289  0.0842992 ]
 [ 0.02888796  0.03510904  0.02128623  0.03419249  0.17759055  0.09398124
   0.02641302  0.15676384  0.16081829  0.26495743]
 [ 0.08124582  0.07019853  0.04374897  0.1714911   0.04523793  0.21604317
   0.05619524  0.05627849  0.13898468  0.12057601]
 [ 0.04306096  0.024569    0.61028194  0.0972095   0.02024116  0.06997558
   0.07006948  0.00616643  0.04925819  0.00916772]
 [ 0.05473436  0.05068987  0.03878     0.09789812  0.04507095  0.15300469
   0.03860227  0.06928228  0.37523344  0.07670398]
 [ 0.01239697  0.10837842  0.22012846  0.0903255   0.05329655  0.07828563
   0.21949193  0.08602186  0.07911382  0.052560

INFO:tensorflow:loss = 1.10009, step = 201 (44.087 sec)
INFO:tensorflow:global_step/sec: 2.26142
INFO:tensorflow:probabilities = [[ 0.0003391   0.00005532  0.00082824  0.00057154  0.13877678  0.00214294
   0.00174891  0.00968066  0.0144812   0.83137524]
 [ 0.95911735  0.00000559  0.01129776  0.00321906  0.00021379  0.01996404
   0.00277036  0.00055318  0.00207598  0.00078284]
 [ 0.01831559  0.01238529  0.02918946  0.0534435   0.00143904  0.01774907
   0.00057161  0.01046518  0.82500714  0.03143413]
 [ 0.01071567  0.00038596  0.00945508  0.7580412   0.00015894  0.15737611
   0.00014447  0.00272364  0.05944404  0.00155501]
 [ 0.00032488  0.60711104  0.0615987   0.0667386   0.01691728  0.02032266
   0.17865346  0.00579169  0.0268645   0.01567719]
 [ 0.01374289  0.00122092  0.01327593  0.00041324  0.74176776  0.02909997
   0.01005171  0.02087094  0.12298589  0.04657066]
 [ 0.00868156  0.08061413  0.00841411  0.01240023  0.05925738  0.09980747
   0.00581605  0.11210046  0.40924758  0.203661

INFO:tensorflow:loss = 0.544384, step = 301 (44.220 sec)
INFO:tensorflow:global_step/sec: 2.23536
INFO:tensorflow:probabilities = [[ 0.00024678  0.01209246  0.00016138  0.02227229  0.01333852  0.00341397
   0.00009135  0.40270439  0.10483406  0.44084468]
 [ 0.19957128  0.00974588  0.15608208  0.09280372  0.03073747  0.07329927
   0.0461236   0.00682714  0.32374951  0.06106007]
 [ 0.00022566  0.00472852  0.00088829  0.01396491  0.0002369   0.01474575
   0.00003967  0.00040158  0.96397746  0.00079134]
 [ 0.00152264  0.00000317  0.99004751  0.0067462   0.00001259  0.00004499
   0.00067526  0.00002075  0.00088409  0.00004291]
 [ 0.92957318  0.00000004  0.04790685  0.00087423  0.00001396  0.00089076
   0.01689098  0.00005187  0.00376461  0.00003359]
 [ 0.00062663  0.43465427  0.00265524  0.09041469  0.02391893  0.0458973
   0.01944458  0.00184709  0.31515241  0.06538889]
 [ 0.00153391  0.00033444  0.00009899  0.00701968  0.01519458  0.12737645
   0.0001237   0.1149196   0.66978335  0.063615

...

INFO:tensorflow:loss = 0.127625, step = 1601 (49.731 sec)
INFO:tensorflow:global_step/sec: 2.02596
INFO:tensorflow:probabilities = [[ 0.99916947  0.          0.00004229  0.00000857  0.          0.00075694
   0.0000135   0.00000134  0.00000761  0.00000037]
 [ 0.00001629  0.02900147  0.01752811  0.04377179  0.04814831  0.00259357
   0.0001023   0.62577718  0.01186678  0.22119416]
 [ 0.0000033   0.00000002  0.00171091  0.99356031  0.00000108  0.00228167
   0.00000002  0.00003867  0.00166602  0.00073807]
 [ 0.00023036  0.98613745  0.00148117  0.00047676  0.00013985  0.00012006
   0.00470974  0.00042814  0.00607835  0.00019813]
 [ 0.00252653  0.00000014  0.03093216  0.00188715  0.00003071  0.00172607
   0.00044211  0.00000019  0.96239698  0.00005799]
 [ 0.00000171  0.00009153  0.99494922  0.00472206  0.          0.00000705
   0.00000022  0.00000007  0.00022821  0.00000001]
 [ 0.00001557  0.00000073  0.00005304  0.00020173  0.00004245  0.00103146
   0.00000039  0.00002165  0.9967705   0.0018

INFO:tensorflow:loss = 0.170559, step = 1701 (49.363 sec)
INFO:tensorflow:global_step/sec: 2.0168
INFO:tensorflow:probabilities = [[ 0.0000856   0.97451723  0.00414391  0.00524411  0.00253292  0.00001062
   0.00057585  0.00956101  0.00288338  0.00044537]
 [ 0.00000541  0.00000217  0.00005792  0.00006717  0.01609668  0.00006709
   0.00000079  0.02497498  0.00017871  0.95854908]
 [ 0.00003737  0.00001508  0.00332964  0.00068943  0.00129982  0.00052895
   0.00007188  0.00000751  0.98625815  0.00776215]
 [ 0.00006482  0.00000002  0.0000004   0.00000662  0.00000595  0.0000076
   0.          0.99606329  0.00000332  0.00384805]
 [ 0.99879634  0.00000016  0.00002158  0.00033572  0.00001827  0.00067196
   0.00000532  0.00002541  0.00000353  0.00012167]
 [ 0.00066352  0.00000338  0.00320537  0.00673644  0.00002013  0.95211798
   0.00265828  0.          0.03459407  0.00000076]
 [ 0.00001922  0.98744464  0.00029137  0.00087076  0.00004941  0.00008717
   0.0000959   0.00020463  0.01066617  0.000270

INFO:tensorflow:loss = 0.0564295, step = 1801 (49.582 sec)
INFO:tensorflow:global_step/sec: 1.7343
INFO:tensorflow:probabilities = [[ 0.99993646  0.          0.0000005   0.00000446  0.00000029  0.00002234
   0.00001171  0.00000034  0.00001906  0.00000493]
 [ 0.00000132  0.00000049  0.00000884  0.99760902  0.00000002  0.002336
   0.00000009  0.00000047  0.0000426   0.0000012 ]
 [ 0.00000037  0.00000189  0.00000418  0.00013916  0.00000014  0.00000036
   0.          0.99974829  0.00001836  0.00008721]
 [ 0.00000112  0.00000073  0.00000252  0.99967515  0.00000009  0.00026786
   0.          0.00000253  0.00001658  0.00003353]
 [ 0.00007183  0.00000101  0.00000861  0.0036583   0.00001909  0.99423224
   0.00000152  0.00028603  0.00165856  0.00006268]
 [ 0.00448421  0.00194779  0.27969486  0.03646516  0.00478052  0.00123527
   0.67101228  0.00000187  0.00036814  0.00000988]
 [ 0.00000183  0.00198405  0.97786456  0.01983936  0.00000267  0.00001231
   0.00007366  0.00000072  0.00021523  0.000005

INFO:tensorflow:loss = 0.100712, step = 1901 (57.678 sec)
INFO:tensorflow:Saving checkpoints for 2000 into /tmp/mnist_convnet_model1c/model.ckpt.
INFO:tensorflow:Loss for final step: 0.13263.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model1c/model.ckpt-2000
------Confusion Matrix------
[[ 961    0    1    1    0    2    9    1    5    0]
 [   0 1109    5    1    0    0    4    1   15    0]
 [   6    0  975   16    2    0    5    9   19    0]
 [   0    1    3  986    0    0    0    6   11    3]
 [   0    0    5    1  944    0   11    3    4   14]
 [   4    1    0   35    2  829   12    2    6    1]
 [   4    2    2    2    3    4  939    0    2    0]
 [   2    3   17    5    2    0    0  975    5   19]
 [   4    0    2   14    4    4    7    4  932    3]
 [   8    6    1   15   18    3    1   10   13  934]]
             precision    recall  f1-score   support

          0       0.97      0.98      0.98       980
          1       0.99      0.98      0.98      1135
   

SystemExit: 

# Model 1D -- Smaller batch size

In [26]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  

  #Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  

In [27]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def get_metrics(a, p):
    actual = a
    predict = p
    print("------Confusion Matrix------")
    print(confusion_matrix(actual, predict))
    
    print(classification_report(actual, predict))
    print("------Accuracy------")
    ascore = accuracy_score(actual, predict)
    print(ascore)

In [28]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model1d")

  # Set up logging for predictions
 #Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)
 
  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=50,
      num_epochs=None,
      shuffle=True)
  
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=2000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    num_epochs = 1,
    shuffle=False)

  results = list(mnist_classifier.predict(input_fn=predict_input_fn))
  predictions = [i['classes'] for i in results]
  get_metrics(eval_labels, predictions)

In [29]:
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9a404c6f10>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_model1d', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model1d/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.08609888  0.10173298  0.11779     0.08528297  0.079

INFO:tensorflow:loss = 2.00755, step = 101 (33.760 sec)
INFO:tensorflow:global_step/sec: 2.41932
INFO:tensorflow:probabilities = [[ 0.0337863   0.00771605  0.00404578  0.0019219   0.58058494  0.06668643
   0.07426667  0.01055099  0.07297433  0.14746653]
 [ 0.00299441  0.03374014  0.0169607   0.18909562  0.00354276  0.00745346
   0.01211778  0.04894644  0.60522681  0.07992186]
 [ 0.00420952  0.00219635  0.00141347  0.01606198  0.13156717  0.01107463
   0.00407064  0.48224697  0.05687457  0.29028463]
 [ 0.0014954   0.47074795  0.04386187  0.01800651  0.02418512  0.01582667
   0.01646474  0.02075299  0.37267208  0.0159867 ]
 [ 0.03397807  0.00482261  0.01211196  0.75334394  0.00207346  0.0835396
   0.01849908  0.0055521   0.04449963  0.0415795 ]
 [ 0.00229113  0.00906394  0.00464117  0.7173174   0.00189276  0.08002772
   0.00863439  0.08402365  0.04141284  0.050695  ]
 [ 0.00325492  0.00128657  0.07582472  0.06421387  0.20082875  0.01078403
   0.57493013  0.03414736  0.01714694  0.0175827

INFO:tensorflow:loss = 0.880903, step = 301 (36.588 sec)
INFO:tensorflow:global_step/sec: 3.08486
INFO:tensorflow:probabilities = [[ 0.01623067  0.00001921  0.00008056  0.02843337  0.00002411  0.71312815
   0.00002053  0.00001453  0.24165641  0.00039239]
 [ 0.00307695  0.00061647  0.00010833  0.00134617  0.01793565  0.01129242
   0.00012661  0.32925966  0.00536783  0.63086981]
 [ 0.00352877  0.00000023  0.0762649   0.02180311  0.00006982  0.00400033
   0.00050962  0.00000799  0.89378095  0.0000343 ]
 [ 0.00952637  0.00180915  0.00132575  0.00341836  0.01565559  0.88695878
   0.00106216  0.0063203   0.04994032  0.0239832 ]
 [ 0.01300988  0.00002169  0.18823163  0.0049659   0.42139152  0.00492925
   0.35190937  0.00080429  0.01234426  0.00239226]
 [ 0.00496383  0.26528645  0.00061857  0.06961308  0.01599442  0.21092886
   0.00261722  0.12783447  0.04345186  0.25869119]
 [ 0.00010519  0.00070014  0.56923205  0.36429131  0.00000149  0.00117832
   0.00019826  0.00002104  0.06426592  0.00000

INFO:tensorflow:loss = 0.574307, step = 501 (34.987 sec)
INFO:tensorflow:global_step/sec: 3.46316
INFO:tensorflow:probabilities = [[ 0.00005783  0.98683923  0.00102301  0.00351675  0.00022905  0.00008218
   0.0004865   0.00428821  0.00133905  0.00213818]
 [ 0.00012545  0.00006713  0.00113808  0.00818635  0.00010062  0.0051629
   0.00003749  0.00000814  0.98444921  0.00072458]
 [ 0.86310643  0.00000002  0.00022752  0.00075587  0.00003044  0.1353378
   0.00022164  0.00001446  0.00026686  0.00003907]
 [ 0.96439618  0.00000003  0.00007158  0.00007321  0.00013532  0.0186188
   0.00021232  0.00001939  0.01589902  0.00057415]
 [ 0.00009538  0.00000677  0.00002042  0.00014828  0.00002441  0.03421976
   0.00000565  0.00004046  0.96509868  0.00034019]
 [ 0.00283724  0.00026131  0.00009807  0.00068428  0.58694941  0.00285119
   0.00276006  0.02024606  0.02452091  0.35879156]
 [ 0.01024968  0.09655059  0.00194684  0.43137291  0.00089257  0.26984787
   0.00144394  0.11311482  0.04128845  0.03329229

INFO:tensorflow:loss = 0.3472, step = 701 (30.349 sec)
INFO:tensorflow:global_step/sec: 2.9149
INFO:tensorflow:probabilities = [[ 0.00017692  0.00000632  0.00800872  0.03802307  0.00133808  0.00182645
   0.00208611  0.00001623  0.94792098  0.00059714]
 [ 0.00575788  0.00001799  0.85560745  0.00563009  0.00350598  0.00110191
   0.11016864  0.00003943  0.01806746  0.00010325]
 [ 0.00405985  0.00075953  0.00060206  0.00169671  0.01316729  0.00054341
   0.0001107   0.0965471   0.1345388   0.74797457]
 [ 0.00015234  0.00000241  0.00000123  0.00155445  0.01822735  0.00011864
   0.00000219  0.00505464  0.00023008  0.9746567 ]
 [ 0.00067121  0.00116131  0.00039685  0.1289916   0.01795826  0.03825571
   0.00154611  0.00273354  0.21042384  0.59786159]
 [ 0.00013053  0.00279307  0.00248673  0.02340666  0.00393286  0.02567325
   0.00020815  0.00160496  0.93424654  0.00551729]
 [ 0.00260333  0.00226374  0.03433464  0.00423435  0.77972919  0.06941108
   0.08979313  0.00038893  0.01608889  0.0011528 

...

INFO:tensorflow:loss = 0.516826, step = 1301 (35.952 sec)
INFO:tensorflow:global_step/sec: 2.74367
INFO:tensorflow:probabilities = [[ 0.00000079  0.0000007   0.00013328  0.99176931  0.00000004  0.00600034
   0.00000001  0.0000069   0.00200681  0.00008191]
 [ 0.00003007  0.00680951  0.00001571  0.00311586  0.00586155  0.00817165
   0.00005407  0.01345642  0.00093107  0.96155405]
 [ 0.99929607  0.00000004  0.00033877  0.00007201  0.00000051  0.00018997
   0.00006641  0.00000068  0.00001716  0.00001846]
 [ 0.00038061  0.00001631  0.06670349  0.89327264  0.00167901  0.00378521
   0.00004657  0.00285389  0.01644891  0.01481327]
 [ 0.0000055   0.00006012  0.00000211  0.00052574  0.03535958  0.00037666
   0.0000146   0.00067784  0.00104925  0.96192861]
 [ 0.00000102  0.00009607  0.99840742  0.00020584  0.00017653  0.00003661
   0.00095993  0.00000017  0.00011612  0.00000031]
 [ 0.00000009  0.00000059  0.00000088  0.00006269  0.000001    0.00000103
   0.          0.98779196  0.00000589  0.0121

INFO:tensorflow:loss = 0.257564, step = 1501 (39.024 sec)
INFO:tensorflow:global_step/sec: 2.60806
INFO:tensorflow:probabilities = [[ 0.00000501  0.00000839  0.00230621  0.00018201  0.00000104  0.00000033
   0.00000003  0.9963851   0.0000142   0.00109776]
 [ 0.00000965  0.00000918  0.99941361  0.00027729  0.00000246  0.00000589
   0.0000138   0.00000027  0.00026776  0.00000012]
 [ 0.00000039  0.00006072  0.00067731  0.03324477  0.00000103  0.00000652
   0.00000005  0.9648059   0.00043235  0.00077088]
 [ 0.00000605  0.00068955  0.00042176  0.00059485  0.03214951  0.0003659
   0.00006265  0.00159284  0.00426258  0.95985425]
 [ 0.00002373  0.95823252  0.00407142  0.00093633  0.00125817  0.00032523
   0.0011713   0.00023149  0.03357478  0.00017514]
 [ 0.00000187  0.00000001  0.00000012  0.00001713  0.00150158  0.00005515
   0.00000037  0.01154253  0.00130139  0.98557985]
 [ 0.00525844  0.00005599  0.00115052  0.00376022  0.0026952   0.00359032
   0.00006173  0.68064106  0.00070243  0.30208

INFO:tensorflow:loss = 0.128514, step = 1701 (26.056 sec)
INFO:tensorflow:global_step/sec: 3.39419
INFO:tensorflow:probabilities = [[ 0.0000029   0.0002578   0.00028878  0.00001161  0.99368918  0.00013032
   0.00043297  0.00005753  0.00222597  0.00290292]
 [ 0.          0.00000014  0.00001442  0.00395981  0.          0.00000006
   0.          0.99577445  0.00000116  0.00024984]
 [ 0.00000014  0.00029113  0.00060229  0.00003359  0.00000174  0.00000158
   0.00000002  0.99830353  0.00022982  0.00053608]
 [ 0.00002509  0.00020788  0.98988008  0.0095601   0.00008006  0.00004842
   0.00010007  0.00006599  0.00002611  0.00000619]
 [ 0.00000012  0.00000115  0.99980778  0.00017263  0.00000009  0.00000009
   0.00000008  0.00001587  0.00000214  0.00000003]
 [ 0.00325777  0.00002093  0.00504691  0.00277904  0.00000369  0.00452767
   0.00003627  0.00000296  0.98430979  0.00001496]
 [ 0.          0.00000297  0.99980325  0.000068    0.00011825  0.00000244
   0.00000027  0.00000001  0.00000427  0.0000

INFO:tensorflow:loss = 0.152459, step = 1901 (26.900 sec)
INFO:tensorflow:Saving checkpoints for 2000 into /tmp/mnist_convnet_model1d/model.ckpt.
INFO:tensorflow:Loss for final step: 0.351253.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model1d/model.ckpt-2000
------Confusion Matrix------
[[ 953    0    2    0    0    1   16    1    6    1]
 [   0 1111    5    1    0    0    4    0   14    0]
 [   4    0 1003    3    1    0    3    4   11    3]
 [   0    2   31  924    0    5    0    3   21   24]
 [   0    0    9    0  912    0    6    0    4   51]
 [   5    1    3   19    1  810   19    1   22   11]
 [   5    2    2    0    3    5  939    0    2    0]
 [   3    7   37    1    4    0    0  841    4  131]
 [   3    0   10    8    4    1    9    2  916   21]
 [   3    6    1    7    4    0    1    1    8  978]]
             precision    recall  f1-score   support

          0       0.98      0.97      0.97       980
          1       0.98      0.98      0.98      1135
  

SystemExit: 

# Final Model
Models 1A - 1D all had a lower accuracy than Model 1. I realized after I ran them that I had accidentally copied over the smaller kernel size (3x3), which had lowered the accuracy in Model 1A relative to model 1. Model 1B had slightly higher accuracy than 1A, so it appears increasing the number of filters did improve the model.

For the final model, I returned the kernel size to 5x5 but changed the padding to "valid," kept the increased number of filters from Model 1B, increased the dropout rate, and used a larger batch size.

This is only a somewhat-educated guess based on the results of the other models.

In [46]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[5, 5],
      padding="valid",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  

  #Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="valid",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 4 * 4 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)
    
  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  

In [43]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def get_metrics(a, p):
    actual = a
    predict = p
    print("------Confusion Matrix------")
    print(confusion_matrix(actual, predict))
    
    print(classification_report(actual, predict))
    print("------Accuracy------")
    ascore = accuracy_score(actual, predict)
    print(ascore)

In [44]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_modelf")

  # Set up logging for predictions
 #Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)
 
  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=200,
      num_epochs=None,
      shuffle=True)
  
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=2000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    num_epochs = 1,
    shuffle=False)

  results = list(mnist_classifier.predict(input_fn=predict_input_fn))
  predictions = [i['classes'] for i in results]
  get_metrics(eval_labels, predictions)

In [47]:
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9a42b305d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_modelf', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_modelf/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.10617886  0.09510051  0.09009317 ...,  0.1001095   0.

INFO:tensorflow:loss = 0.221374, step = 1101 (124.866 sec)
INFO:tensorflow:global_step/sec: 0.73618
INFO:tensorflow:probabilities = [[ 0.00057361  0.00606381  0.03949743 ...,  0.91211069  0.01334902
   0.0179261 ]
 [ 0.00000168  0.00000043  0.99874866 ...,  0.00000571  0.00000154
   0.00000005]
 [ 0.00021368  0.0000066   0.00678453 ...,  0.00860117  0.00013275
   0.40532601]
 ..., 
 [ 0.00237135  0.00001501  0.00234526 ...,  0.00000548  0.00006584
   0.00003706]
 [ 0.00002961  0.99336278  0.00112821 ...,  0.00186636  0.00060161
   0.00143779]
 [ 0.05790169  0.03144976  0.08900544 ...,  0.00418706  0.41204935
   0.00756569]] (135.837 sec)
INFO:tensorflow:loss = 0.287053, step = 1201 (135.836 sec)
INFO:tensorflow:global_step/sec: 0.797514
INFO:tensorflow:probabilities = [[ 0.00020306  0.00008753  0.00075605 ...,  0.00000011  0.95887482
   0.00000667]
 [ 0.00002072  0.00000057  0.99380672 ...,  0.00066153  0.0008449
   0.00068494]
 [ 0.01252133  0.00021007  0.01795153 ...,  0.00724055  0.

SystemExit: 

This model is still slightly less accurate than the first model (96.51% vs 96.99%) and takes significantly longer to run.

# Questions
### How many hidden layers are in your network?
Model 1: 2 convolution layers + 2 activation layers + 2 pooling layers + 1 dense layer + 1 dropout layer + 1 logits layer = 9 hidden layers

Model 2: 1 convolution layer + 1 activation layer + 1 pooling layer + 1 dense layer + 1 dropout layer + 1 logits layer = 6 hidden layers

Model 3: 2 convolution layers + 2 activation layers + 1 pooling layer + 1 dense layer + 1 dropout layer + 1 logits layer = 8 hidden layers

I played with the hyperparameters in each model, so it's not a direct comparison, but it seems like getting rid of a pooling layer (in model 3, compared to model 1) did not have a significant effect on accuracy, but it did slow down the training significantly.

Models 1A - 1D and final model: same as model 1 (9 hidden layers)

### How many convolutions are calculated in each convolution layer?
The number of calculations in each convolution layer is affected by the size of the kernel, the size of the image, and the number of filters.

For the best of the original three models (Model 1), both convolution layers have 5x5 kernels and "same" padding. The images are 28 x 28 going into conv1, and 14 x 14 going into conv2. Conv1 has 28 x 28 convolutions for each filter and 32 filters, which makes 25,088 convolutions. Conv2 has 14 x 14 convolutions for each filter and 64 filters, which makes 12,544 convolutions.
### Run a single test example through the model and print some of the convolved images from the first layer. Can you see any features from the image that are revealed by printing?

When I first turned in the assignment, I was struggling with how to run a single test example through the model. The Tensorflow estimator doesn't allow access to the tensors in a way that would let me print some of the convolved images. I tried grabbing this code from one of the notebooks on Github:

import matplotlib.pyplot as plt  
def reset_graph(seed=42):  
    tf.reset_default_graph()  
    tf.set_random_seed(seed)  
    np.random.seed(seed)
    

x = tf.placeholder(tf.float32, shape=None, name = "x")  
h = tf.placeholder(tf.float32, shape=None, name = "h")  

init = tf.global_variables_initializer()

with tf.Session() as sess:  
    sess.run(init)  
    result = sess.run(conv1, feed_dict =   
                      {x: inp, h: filt})  
    print(result.shape)  
    plt.gray()   
    plt.matshow(result.reshape(26,26))   
    plt.show()
    
#### I rebuilt the most successful model without the estimator, so that I could run the test example below:

In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
X_trn = mnist.train.images  # Returns np.array
y_trn = np.asarray(mnist.train.labels, dtype=np.int32)
X_test = mnist.test.images  # Returns np.array
y_test = np.asarray(mnist.test.labels, dtype=np.int32)

# placeholders
X = tf.placeholder(tf.float32, shape=[None, 784], name = "X")
y = tf.placeholder(tf.int32, shape=[None], name="y")

X_flat = tf.reshape(X, [-1, 28, 28, 1])

# first conv layer
conv1 = tf.layers.conv2d(X_flat, filters=32, kernel_size=[5, 5], strides=[1, 1], padding = 'SAME')

# first pool layer
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

# second conv layer
conv2 = tf.layers.conv2d(conv1, filters=64, kernel_size=[5, 5], strides=[1, 1], padding = 'SAME')

# second pool layer
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

# flatten pooling results
pool2_shape = pool2.get_shape().as_list()
pool2_flat = tf.reshape(pool2, shape=[-1, pool2_shape[1]*pool2_shape[2]*pool2_shape[3]])

# dense layer
dense = tf.layers.dense(pool2_flat, units=1024, activation=tf.nn.relu)

# dropout layer
dropout = tf.layers.dropout(inputs=dense, rate=0.4)

# logits
t = tf.layers.dense(inputs=dropout, units=10, name = "t", activation=tf.nn.relu)

# softmax score
y_prob = tf.nn.softmax(t, name="y_prob")

predicted = tf.cast(tf.argmax(tf.transpose(y_prob)), tf.int32)

correct = tf.equal(predicted, y)

accuracy = tf.reduce_mean(tf.cast(correct, "float"))

# training parts of the graph
onehot_labels = tf.one_hot(indices=tf.cast(y, tf.int32), depth=10)
xentropy = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=y_prob)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
training_op = optimizer.minimize(
        loss=tf.reduce_mean(xentropy),
        global_step=tf.train.get_global_step())


init = tf.global_variables_initializer()

# saver = tf.train.Saver()  # saves the model.  not strictly necessary.

with tf.Session() as sess:
    sess.run(init)
   
    for _ in range(2000):
        X_batch, y_batch = mnist.train.next_batch(1)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = correct.eval(feed_dict={X: X_batch, y: y_batch})

        # print every 10
        if _ % 10 == 0:
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(_, "Train accuracy:", sum(acc_train)/100, "Test accuracy:", acc_test)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Tensor("y_prob:0", shape=(?, 10), dtype=float32)
Tensor("Equal:0", shape=(?,), dtype=bool)
Tensor("Mean:0", shape=(), dtype=float32)
0 Train accuracy: 0.0 Test accuracy: 0.0907
10 Train accuracy: 0.0 Test accuracy: 0.1152
20 Train accuracy: 0.0 Test accuracy: 0.1108
30 Train accuracy: 0.0 Test accuracy: 0.098
40 Train accuracy: 0.0 Test accuracy: 0.1408
50 Train accuracy: 0.0 Test accuracy: 0.0947
60 Train accuracy: 0.01 Test accuracy: 0.1631
70 Train accuracy: 0.0 Test accuracy: 0.1422
80 Train accuracy: 0.01 Test accuracy: 0.1319
90 Train accuracy: 0.01 Test accuracy: 0.1503
100 Train accuracy: 0.01 Test accuracy: 0.1557
110 Train accuracy: 0.01 Test accuracy: 0.2223
120 Train accuracy: 0.0 Test accuracy: 0.2341
130 Train accuracy: 0.01 Test accuracy: 0.2564
140 Train accuracy: 0.01 Test accur

KeyboardInterrupt: 

In [12]:
import matplotlib.pyplot as plt
conv1_output = sess.run(conv1, feed_dict={X:X_test[0]}
print(conv1_output.shape)
plt.gray()
plt.matshow(conv1_output.reshape(26,26))
plt.show()

SyntaxError: invalid syntax (<ipython-input-12-c0ebf7a1da03>, line 3)

Unfortunately, the model above was taking too long to run (I left it going for half the night and it didn't finish; I'm not sure what the issue was since this hasn't been a problem for the other models), and I wanted to make sure I got this turned in. The code in the cell above is what I would have used to view the convoluted images if the model had finished.

In general, some features we've seen highlighted after convolutions include edge detection and emphasis of contrast.

### Print couple of the convolution kernels as matrices (no need to print images). What kind of patterns can you see in the convolution kernels?

I had the same issues with this question as I did with the one before it, in both my first and second attempts at this assignment. Here is the code I intended to run to view the kernels as matrices if the model had finished:


In [10]:
gr = tf.get_default_graph()  
conv1_kernel0_val = gr.get_tensor_by_name('conv1/kernel:0').eval()  
conv1_kernel10_val = gr.get_tensor_by_name('conv1/kernel:10').eval()  
conv1_kernel30_val = gr.get_tensor_by_name('conv1/kernel:30').eval()  

print (conv1_kernel0_val, conv1_kernel10_val, conv1_kernel30_val)  

KeyError: "The name 'conv1/kernel:0' refers to a Tensor which does not exist. The operation, 'conv1/kernel', does not exist in the graph."

### JD-comments

Great effort Athena. I think you are on the right track for getting the intermediate layers to print. I also like your exposition of the model as you go.  This is good work.

I didnt build any code to print intermediate layers using the Estimation API.  In regular tensorflow, using sessions, it is trivial to do this.  There are examples [here](https://github.com/JoeDumoulin/CSCD439F17/blob/master/notebooks/Convolutional%20Networks/Intro%20to%20Convolution.ipynb).

For keras, you can see the example [here](https://github.com/JoeDumoulin/CSCD439F17/blob/master/notebooks/Convolutional%20Networks/Convolutional%20Network%20for%20MNIST%20in%20Keras.ipynb)